<a href="https://colab.research.google.com/github/karoldem/minicalendar/blob/miniatures.ipynb/miniatures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from googleapiclient.discovery import build
from google.oauth2 import service_account
import json
import cv2
import numpy as np
import requests
from googleapiclient.http import MediaFileUpload
from google.colab import userdata

# Setup
creds = service_account.Credentials.from_service_account_info(
    json.loads(userdata.get('slidesAPI')),
    scopes=["https://www.googleapis.com/auth/drive", "https://www.googleapis.com/auth/presentations"]
)
drive_service = build("drive", "v3", credentials=creds)
slides_service = build("slides", "v1", credentials=creds)

PRESENTATION_ID = '16XcadkghKtzrdaywqfpLGV-p1jkqjWCFY8RG9PelPKg'
presentation = slides_service.presentations().get(presentationId=PRESENTATION_ID).execute()

# Get image data
images = []
for slide in presentation.get('slides', []):
    for element in slide.get('pageElements', []):
        if 'image' in element:
            images.append({
                'contentUrl': element['image']['contentUrl'],
                'sourceUrl': element['image'].get('sourceUrl'),
                'crop': element['image'].get('imageProperties', {}).get('cropProperties', {}),
                'transform': element['transform'],
                'size': element['size'],
                'pageId': slide['objectId']
            })

# Download image
resp = requests.get(images[0]['contentUrl'])
img_array = np.frombuffer(resp.content, np.uint8)
img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)

# Crop
h, w = img.shape[:2]
left = images[0]['crop'].get('leftOffset', 0)
right = images[0]['crop'].get('rightOffset', 0)
top = images[0]['crop'].get('topOffset', 0)
bottom = images[0]['crop'].get('bottomOffset', 0)

x1 = int(w * left)
x2 = int(w * (1 - right))
y1 = int(h * top)
y2 = int(h * (1 - bottom)) if bottom else h

cropped = img[y1:y2, x1:x2]

# Save cropped to file
cv2.imwrite("cropped.jpg", cropped)

# Upload to Drive
file_metadata = {'name': 'cropped.jpg', 'mimeType': 'image/jpeg'}
media = MediaFileUpload('cropped.jpg', mimetype='image/jpeg')
uploaded = drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()
file_id = uploaded.get('id')

# Make file public
drive_service.permissions().create(
    fileId=file_id,
    body={"role": "reader", "type": "anyone"},
).execute()

# Build URL
image_url = f"https://drive.google.com/uc?export=view&id={file_id}"

# Insert into Slides
requests_body = [
    {
        "createImage": {
            "url": image_url,
            "elementProperties": {
                "pageObjectId": images[0]['pageId'],
                "size": images[0]['size'],
                "transform": images[0]['transform']
            }
        }
    }
]

slides_service.presentations().batchUpdate(
    presentationId=PRESENTATION_ID,
    body={'requests': requests_body}
).execute()

# Delete uploaded image
drive_service.files().delete(fileId=file_id).execute()


''